<a href="https://colab.research.google.com/github/tiagopint0/Trabalho2PCD/blob/main/Ficha1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using HuggingFaces

## Raw ```pipeline``` version

Veja que é preciso definir o *downstream task* pretendido.

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier([
        "I've been waiting for a HuggingFace course my whole life.",
        "I hate this so much!",
    ])

In [ ]:
from transformers import pipeline

classifier = pipeline(model="FacebookAI/roberta-large-mnli")
classifier([
        "I've been waiting for a HuggingFace course my whole life.",
        "I hate this so much!",
    ])

In [ ]:
from transformers import pipeline

classifier = pipeline('sentiment-analysis', model="FacebookAI/roberta-large-mnli")
classifier([
        "I've been waiting for a HuggingFace course my whole life.",
        "I hate this so much!",
    ])

Some weights of the model checkpoint at FacebookAI/roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'label': 'NEUTRAL', 'score': 0.7374850511550903},
 {'label': 'NEUTRAL', 'score': 0.5799964666366577}]

## Featured version

### 1. Model input

Veja que este é um modelo *fine tuned*.

In [ ]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

raw_inputs = ["I've been waiting for a HuggingFace course my whole life.", "I hate this so much!",]

inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")

In [ ]:
print(inputs)
print(inputs['input_ids'].size())

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}
torch.Size([2, 16])


### 2. Task Selection

Selecione a classe com base no *downstream task* pretendido. O ```AutoModel``` irá sempre retornar *hidden states*.

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)

In [ ]:
print(outputs.logits) #Logits are raw scores outputted by the last layer of the model.
print(outputs.logits.shape)

tensor([[-1.5607,  1.6123],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>)
torch.Size([2, 2])


### 3. Model Output

In [ ]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1) #To the logits be converted to probabilities, they need to go through a SoftMax layer
print(predictions)

tensor([[4.0195e-02, 9.5980e-01],
        [9.9946e-01, 5.4418e-04]], grad_fn=<SoftmaxBackward0>)


In [ ]:
print(model.config.id2label)
[{'label': model.config.id2label[0], 'score': float(p[0])} if p[0] > p[1] else {'label': model.config.id2label[1], 'score': float(p[1])}
  for p in predictions]

{0: 'NEGATIVE', 1: 'POSITIVE'}


[{'label': 'POSITIVE', 'score': 0.9598048329353333},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

## Exercício

1. Consulte o endereço https://huggingface.co/models. Liste cinco tarefas que podem ser encontradas e três línguas que podem ser usadas para filtrar os modelos disponíveis.
1. Ainda no endereço https://huggingface.co/models. Coloque o link que permita acessar o BERT, o GPT2 e o Llama.

Considere o review:
> Dear Amazon, last week I ordered an Optimus Prime action figure
from your online store in Germany. Unfortunately, when I opened the package,
I discovered to my horror that I had been sent an action figure of Megatron
instead! As a lifelong enemy of the Decepticons, I hope you can understand my
dilemma. To resolve the issue, I demand an exchange of Megatron for the
Optimus Prime figure I ordered. Enclosed are copies of my records concerning
this purchase. I expect to hear from you soon. Sincerely, Bumblebee.

3. Usando o ```pipeline```
   1. classifique o sentimento do review.
   1. extraia o NER do review.
   1. gere uma resposta para o review.

4. **Desafio:** escolha um texto em *Português* e aplique sobre ele três tarefas que façam sentido.

1.

5 tarefas que são possíveis:

* Text Generation
* Question Answering
* Feature Extraction
* Summarization
* Text Classification

Linguagens:

* Portuguese
* Greek
* Chinese

2.

* GPT2: https://huggingface.co/openai-community/gpt2
* Llama: https://huggingface.co/meta-llama/Llama-2-7b
* Bert: https://huggingface.co/google-bert/bert-base-uncased

3.

3.1

In [1]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis")
classifier([
  "Dear Amazon, last week I ordered an Optimus Prime action figure from your online store in Germany. Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action figure of Megatron instead! As a lifelong enemy of the Decepticons, I hope you can understand my dilemma. To resolve the issue, I demand an exchange of Megatron for the Optimus Prime figure I ordered. Enclosed are copies of my records concerning this purchase. I expect to hear from you soon. Sincerely, Bumblebee."
])



No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

[{'label': 'NEGATIVE', 'score': 0.9015458226203918}]

3.2

In [7]:
from transformers import pipeline

def extract_ner(input_string):
    # Initialize the Named Entity Recognition pipeline
    pipe = pipeline('ner', model='dslim/bert-base-NER')

    # Use the pipeline to extract entities from the input string
    results = pipe(input_string)
    # Format the results as a list of tuples
    entities = [(ent['word'], ent['entity']) for ent in results]

    return entities

# Test the function with some sample text
entities = extract_ner("Dear Amazon, last week I ordered an Optimus Prime action figure from your online store in Germany. Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action figure of Megatron instead! As a lifelong enemy of the Decepticons, I hope you can understand my dilemma. To resolve the issue, I demand an exchange of Megatron for the Optimus Prime figure I ordered. Enclosed are copies of my records concerning this purchase. I expect to hear from you soon. Sincerely, Bumblebee.")
for word, entity in entities:
    print(f"{word}: {entity}")

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Amazon: B-ORG
Op: B-MISC
##ti: B-MISC
##mus: I-MISC
Prime: I-MISC
Germany: B-LOC
Mega: B-PER
##tron: I-MISC
Dec: B-MISC
##ept: I-MISC
##ico: I-MISC
##ns: I-MISC
Mega: B-PER
##tron: I-ORG
Op: B-MISC
##ti: B-MISC
##mus: I-MISC
Prime: I-MISC
B: B-PER
##umble: I-MISC


3.3

In [9]:
def generate_response(review):
    # Initialize the Sentiment Analysis pipeline
    sent_pipe = pipeline('sentiment-analysis', model='cardiffnlp/twitter-roberta-base-sentiment-latest')

    # Detect the sentiment of the review
    sentiment_result = sent_pipe(review)[0]
    sentiment = sentiment_result['label'].lower()

    # Generate a response based on the detected sentiment
    if sentiment == 'positive':
        response = "Thanks for leaving a positive review! We appreciate your support."
    elif sentiment == 'negative':
        response = "Sorry to hear that! Please let us know what went wrong and how we can fix it."
    else:
        response = "We're not sure how to interpret your review. Could you please clarify your thoughts?"

    return response

# Sample review
review =   "Dear Amazon, last week I ordered an Optimus Prime action figure from your online store in Germany. Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action figure of Megatron instead! As a lifelong enemy of the Decepticons, I hope you can understand my dilemma. To resolve the issue, I demand an exchange of Megatron for the Optimus Prime figure I ordered. Enclosed are copies of my records concerning this purchase. I expect to hear from you soon. Sincerely, Bumblebee."

# Call the function to generate a response
response = generate_response(review)

print(response)


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Sorry to hear that! Please let us know what went wrong and how we can fix it.


4.

In [20]:
review_sushi = "A minha experiência no Sushimix foi desapontante. O ambiente era pouco cativante e os pratos não estavam muito frescos, tinham uma apresentação algo descuidada. O serviço foi rápido, mas não com a melhor qualidade. Não sei se volto."


#translation
from transformers import pipeline

def translate_to_english(portuguese_sentence):
    trans_pipe = pipeline('translation_porft_to_en', model='unicamp-dl/translation-pt-en-t5')
    translated_sentence = trans_pipe(portuguese_sentence, max_length = 400)[0]['translation_text']
    return translated_sentence

portuguese_sentence = review_sushi
translated_sentence = translate_to_english(portuguese_sentence)
print(translated_sentence)

print("\n")
# Sentiment Analysis
classifier = pipeline("sentiment-analysis")
classifier([review_sushi])


#summarization
from transformers import pipeline

def summarize_article(article):
    # Initialize the summarization pipeline
    summ_pipe = pipeline('summarization', model='t5-small')

    # Summarize the article
    summary = summ_pipe(article, min_length=30, max_length=60)[0]['summary_text']

    return summary

summary = summarize_article(review_sushi)

summary


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


My experience in the Sushimix was disappointing.The environment was little captivating and the dishes were not very fresh, they had a presentation that was somewhat neglected.The service was fast, but not with the best quality.I don’t know if I go back.




'a minha experiência no Sushimix foi desapontante . o ambiente era pouco cativante e os pratos no estavam muito frescos .'